In [ ]:
import requests
import pandas as pd
import time

In [ ]:
class RateLimiter:
    def __init__(self, per_second=3, per_minute=60):
        self.per_second = per_second
        self.per_minute = per_minute
        self.requests_made_second = 0
        self.requests_made_minute = 0
        self.start_time = time.time()

    def wait_for_second(self):
        current_time = time.time()
        elapsed_time = current_time - self.start_time
        if elapsed_time < 1:
            self.requests_made_second += 1
            if self.requests_made_second >= self.per_second:
                time.sleep(1 - elapsed_time)
                self.requests_made_second = 0
                self.start_time = time.time()

    def wait_for_minute(self):
        current_time = time.time()
        elapsed_time = current_time - self.start_time
        if elapsed_time < 60:
            self.requests_made_minute += 1
            if self.requests_made_minute >= self.per_minute:
                time.sleep(60 - elapsed_time)
                self.requests_made_minute = 0
                self.start_time = time.time()

In [ ]:
def fetch_data_with_retry(endpoint_url, rate_limiter, max_retries=3):
    retries = 0
    while retries < max_retries:
        rate_limiter.wait_for_second()
        rate_limiter.wait_for_minute()
        response = requests.get(endpoint_url)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:  # Rate limit exceeded
            print("Rate limit exceeded. Waiting before retrying...")
            time.sleep(10)
            retries += 1
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            return None
    print("Max retries reached. Unable to fetch data.")
    return None

In [ ]:
df = pd.read_json("data/data.json")

In [ ]:
df.head()

In [ ]:
def fetch_data_with_retry(endpoint_url, rate_limiter, max_retries=3):
    retries = 0
    while retries < max_retries:
        rate_limiter.wait_for_second()
        rate_limiter.wait_for_minute()
        response = requests.get(endpoint_url)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:  # Rate limit exceeded
            print("Rate limit exceeded. Waiting before retrying...")
            time.sleep(10)
            retries += 1
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            return None
    print("Max retries reached. Unable to fetch data.")
    return None

In [ ]:
column_names = ["mal_id", "mal_id_recomm", "votes"]
df1 = pd.DataFrame(columns=column_names)

In [ ]:
base_url = "https://api.jikan.moe/v4/anime/"
limiter = RateLimiter()

In [ ]:
for mal_id in df['mal_id']:
    endpoint_url = f"{base_url}{mal_id}/recommendations"
    data = fetch_data_with_retry(endpoint_url, limiter)
    if data['data']:
        for entry in data['data']:
            new_entry = {'mal_id' : mal_id, 'mal_id_recomm': entry['entry']['mal_id'], 'votes' : entry['votes']}
            df1 = df1._append(new_entry, ignore_index=True)

In [ ]:
df1['mal_id_recomm'] = df3['mal_id_recomm'].astype(int)
df1['votes'] = df3['votes'].astype(int)

In [ ]:
df1.to_csv('data/user_recommendation.csv', index=False)